In [2]:
%pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [1]:
import json

class JsonModelParser:
    def __init__(self, filepath):
        self.filepath = filepath
        self.data = self.load_json_file()
        self.parse_model_data()

    def printDetails(self):
        print("\tINPUT SIZE:",self.input_size)
        print("\tLAYERS:", self.layers)
        print("\tINPUT:", self.input)
        print("\tINITIAL WEIGHTS:", self.initial_weights)
        print("\tTARGET:", self.target)
        print("\tLEARNING RATE:", self.learning_rate)
        print("\tBATCH SIZE:", self.batch_size)
        print("\tMAX ITERATION:", self.max_iteration)
        print("\tERROR THRESHOLD:", self.error_threshold)
    

    def load_json_file(self):
        try:
            with open(self.filepath, 'r', encoding='utf-8') as file:
                return json.load(file)
        except FileNotFoundError:
            print(f"The file {self.filepath} was not found")
            return None
        except json.JSONDecodeError:
            print(f"Error decoding JSON from the file {self.filepath}")
            return None

    def parse_model_data(self):
        if self.data:
            self.case = self.data.get('case', {})
            self.model = self.case.get('model', {})
            self.input_size = self.model.get('input_size')

            raw_layers = self.model.get('layers', [])   
            self.layers = [{'number_of_neurons': layer.get('number_of_neurons'),
                        'activation_function': layer.get('activation_function')}
                       for layer in raw_layers]
            
            self.input = self.case.get('input', [])
            print(self.case.get("initial_weights"))
            self.initial_weights = self.case.get('initial_weights', [])
            self.target = self.case.get('target', [])
            self.parameters = self.case.get('learning_parameters', {})

            self.learning_rate = self.parameters.get('learning_rate')
            self.batch_size = self.parameters.get('batch_size')
            self.max_iteration = self.parameters.get('max_iteration')
            self.error_threshold = self.parameters.get('error_threshold')

            self.expect = self.data.get('expect', {})
            self.stopped_by = self.expect.get('stopped_by', '')
            self.final_weights = self.expect.get('final_weights', [])

    @staticmethod
    def save_json_file(data, filepath):
        try:
            with open(filepath, 'w', encoding='utf-8') as file:
                json.dump(data, file, ensure_ascii=False, indent=4)
        except IOError:
            print(f"Could not save data to {filepath}")

In [54]:
import tensorflow as tf

class TensorFlowModel: 
    def __init__(self, inputs, targets, learning_rate, batch_size, initial_weights, layers) -> None:
        self.inputs = tf.constant(inputs, dtype=tf.float32)
        self.targets = tf.constant(targets, dtype=tf.float32)
        self.initial_weights = tf.constant(initial_weights, dtype=tf.float32)
        self.input_size = len(inputs[0])
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.dataset = tf.data.Dataset.from_tensor_slices((self.inputs, self.targets))
        self.dataset = self.dataset.batch(batch_size)
        self.model = tf.keras.Sequential()
        for i in range(len(layers)):
            current_layer = layers[i]
            current_number_of_neurons = current_layer["number_of_neurons"]
            current_activation = current_layer["activation_function"]
            self.model.add(tf.keras.layers.Dense(
                current_number_of_neurons, 
                input_shape = (self.input_size,),
                activation = current_activation, 
                use_bias=True,
                bias_initializer=tf.keras.initializers.Constant(1.0)
            ))
        for i in range(len(initial_weights)):
            number_of_neurons = layers[i]["number_of_neurons"]
            self.model.layers[i].set_weights([self.initial_weights[i], tf.ones((number_of_neurons,),dtype=tf.float32)])
        # self.model.layers[0].set_weights([self.initial_weights])

    def train_step(self, inputs, targets):
        optimizer = tf.keras.optimizers.SGD(learning_rate=self.learning_rate)
        print(self.model.layers[-1].activation)
        loss_function = tf.keras.losses.CategoricalCrossentropy() if self.model.layers[-1].activation == tf.keras.activations.softmax else tf.keras.losses.MeanSquaredError()
        print(f"Ini loss_function : {loss_function}")
        with tf.GradientTape() as tape:
            predictions = self.model(inputs, training=True)
            loss = loss_function(targets, predictions)

        gradients = tape.gradient(loss, self.model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, self.model.trainable_variables))
        return loss

    def show_prediction(self):
        print("========================================")
        print("Model layers:", self.model.layers)
        updated_weight = self.model.layers[0].get_weights()
        print("Updated weights:\n", updated_weight)
        print("=========================================")

    def predict(self, max_iteration):
        for i in range(max_iteration):
            for inputs_batch, targets_batch in self.dataset:
                loss = self.train_step(inputs_batch, targets_batch)
            print(f'Iteration {i+1}, Loss: {loss.numpy()}')

        self.show_prediction()


In [56]:
import os

def main():
    currDir = os.getcwd()
    testDir = currDir.replace("src","test")
    file = os.path.join(testDir, "mlp.json")
    json_file = JsonModelParser(file)
    inputs = json_file.input
    print("ini inputs : ", inputs)
    targets = json_file.target
    learning_rate = json_file.learning_rate
    layers = json_file.layers
    batch_size = json_file.batch_size
    initial_weights = json_file.initial_weights
    max_iter = json_file.max_iteration
    model = TensorFlowModel(inputs, targets, learning_rate, batch_size, initial_weights, layers)
    model.predict(max_iter)

main()

[[[0.1, 0.2], [-0.3, 0.5], [0.4, 0.5]], [[0.2, 0.1], [0.4, -0.5], [0.7, 0.8]]]
ini inputs :  [[-1.0, 0.2], [0.2, -1.0]]


ValueError: Layer dense_53 weight shape (2, 2) is not compatible with provided weight shape (3, 2).

In [14]:
import tensorflow as tf

# Define the inputs and targets
inputs = tf.constant([
    [3.0, 1.0],
    [1.0, 2.0]
], dtype=tf.float32)

targets = tf.constant([
    [2.0, 0.3, -1.9],
    [1.3, -0.7, 0.1]
], dtype=tf.float32)

# Number of neurons, activation, and learning rate
number_of_neurons = 3
activation = "linear"
learning_rate = 0.1
batch_size = 2  # as you mentioned, batch_size of 2

# Initial weights and new uniform bias of 1
initial_weights = [
    [0.1, 0.3, 0.2],  # weights for inputs to neuron 1
    [0.4, 0.2, -0.7]  # weights for inputs to neuron 2
]
initial_bias = [1.0, 1.0, 1.0]  # uniform bias for all neurons

# Create the model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(number_of_neurons, activation=activation, input_shape=(2,), use_bias=True)
])

# Set the initial weights and bias
model.layers[0].set_weights([tf.constant(initial_weights, dtype=tf.float32), tf.constant(initial_bias, dtype=tf.float32)])

# Compile the model with the specified optimizer and learning rate
optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='mse')

# Since this is a simple forward pass, we'll use predict instead of fit to demonstrate batching
outputs = model.predict(inputs, batch_size=batch_size)

print("Outputs:")
print(outputs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Outputs:
[[ 1.7         2.1         0.90000004]
 [ 1.9000001   1.7        -0.19999993]]
